# Resnet Inflate

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchinfo import summary
from tqdm import tqdm
import matplotlib.pyplot as plt

from local import data
from local import resnet
from local import train
from local.device import device, cpu


print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)
print("Pytorch device: ", device)


/opt/anaconda3/envs/network_inflation/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/anaconda3/envs/network_inflation/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /opt/anaconda3/envs/network_inflation/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/network_inflation/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/opt/anaconda3/envs/network_inflation/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/opt/anaconda3/envs/network_inflation/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/opt/anaconda3/envs/network_inflation/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/opt/anaconda3/envs/network_infl

PyTorch Version:  1.13.1
Torchvision Version:  0.14.1a0
Pytorch device:  <function device at 0x163619cf0>


In [2]:
train_data = data.load_dataset("data/imagenet-1k/train/", transform=data.train_transform())
train_loader = torch.utils.data.DataLoader2(
    train_data,
    batch_size=8,
    shuffle=True,
    collate_fn=data.train_collate_fn(train_data)
)

Loading data `data/imagenet-1k/train/`... DONE


In [5]:
_, resnet50 = resnet.network_load("resnet50", reset=False)

Using cache found in /Users/marakim/.cache/torch/hub/pytorch_vision_v0.10.0


In [6]:
_, resnet101_inflate = resnet.network_load("resnet101", "resnet50", reset=True)

Using cache found in /Users/marakim/.cache/torch/hub/pytorch_vision_v0.10.0


In [7]:
resnet50 = resnet50.eval()
resnet101_inflate = resnet101_inflate.eval()

In [8]:
module_input = {}
module_output = {}
def get_io(name):
    def hook(model, input, output):
        module_input[name] = input
        module_output[name] = output.detach()
    return hook

In [26]:
resnet50.conv1.register_forward_hook(get_io('conv1'))
resnet50.bn1.register_forward_hook(get_io('bn1'))
resnet50.layer1[0].register_forward_hook(get_io('layer1[0]'))
resnet50.layer1[1].register_forward_hook(get_io('layer1[1]'))
resnet50.layer3[0].register_forward_hook(get_io('layer3[0]'))

In [27]:
resnet101_inflate.conv1.register_forward_hook(get_io('conv1'))
resnet101_inflate.bn1.register_forward_hook(get_io('bn1'))
resnet101_inflate.layer1[0].register_forward_hook(get_io('layer1[0]'))
resnet101_inflate.layer1[1].register_forward_hook(get_io('layer1[1]'))
resnet101_inflate.layer3[0].register_forward_hook(get_io('layer3[0]'))

In [11]:
example_input, example_label = next(iter(train_loader))
example_input.shape

torch.Size([8, 3, 224, 224])

In [28]:
resnet50(example_input)
module_input['layer3[0]'][0][0,0]

tensor([[8.2152e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [29]:
resnet101_inflate(example_input)
module_input['layer3[0]'][0][0,0]

tensor([[8.2152e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

In [18]:
resnet50.bn1.state_dict()
resnet101_inflate.bn1.state_dict()

OrderedDict([('weight',
              tensor([1.5015, 2.3959, 2.4188, 2.1209, 4.6100, 2.0853, 1.6228, 2.1865, 2.7581,
                      2.0903, 1.7681, 2.1057, 6.8309, 2.6507, 1.9449, 2.3006, 1.7701, 2.2712,
                      1.6792, 7.5183, 2.3296, 1.7288, 2.3918, 2.2404, 5.9957, 2.2425, 2.5086,
                      2.2304, 1.5343, 9.4247, 1.7311, 2.0335, 6.2795, 2.2514, 2.3651, 2.1595,
                      1.7528, 2.0686, 1.7344, 7.1874, 2.2505, 4.7094, 1.8000, 1.7114, 1.8585,
                      1.6867, 1.7534, 1.8077, 1.5146, 2.0271, 1.4854, 1.4940, 4.4586, 1.5751,
                      2.6822, 1.9343, 2.1422, 2.4709, 2.5950, 2.4721, 1.6491, 2.3366, 1.4622,
                      1.7303])),
             ('bias',
              tensor([ 3.0887,  2.2352,  4.1188,  1.4539, -4.5286,  2.6051,  2.7197,  2.4477,
                       1.1948,  2.0417,  2.7164,  6.4281, -6.5246,  1.1299,  2.5385,  5.4271,
                       4.7583, 10.0577,  2.9221, 14.8767,  2.5501,  2.6791,

In [7]:
resnet50.get_parameter('conv1.weight')[0, 0]

tensor([[ 0.0181,  0.0295,  0.0026, -0.0224, -0.0101, -0.0734,  0.0227],
        [ 0.0019, -0.0354,  0.0323,  0.0115, -0.0024, -0.0357, -0.0056],
        [-0.0291, -0.0287,  0.0239,  0.0026, -0.0302, -0.0470, -0.0023],
        [-0.0210, -0.0114,  0.0041,  0.0169,  0.0041, -0.0070, -0.0045],
        [ 0.0273, -0.0037,  0.0130, -0.0513, -0.0063,  0.0126, -0.0177],
        [-0.0241,  0.0179, -0.0275, -0.0067,  0.0005,  0.0696, -0.0434],
        [-0.0354, -0.0083,  0.0192, -0.0295,  0.0320, -0.0178, -0.0417]],
       grad_fn=<SelectBackward0>)

In [8]:
with torch.no_grad():
    resnet50.get_parameter('conv1.weight')[0, 0, 0, 0].copy_(7)

In [9]:
resnet50.get_parameter('conv1.weight')[0, 0]

tensor([[ 7.0000e+00,  2.9486e-02,  2.5879e-03, -2.2356e-02, -1.0119e-02,
         -7.3394e-02,  2.2696e-02],
        [ 1.8616e-03, -3.5401e-02,  3.2312e-02,  1.1514e-02, -2.3707e-03,
         -3.5690e-02, -5.5990e-03],
        [-2.9055e-02, -2.8673e-02,  2.3896e-02,  2.5682e-03, -3.0179e-02,
         -4.7050e-02, -2.2803e-03],
        [-2.1034e-02, -1.1398e-02,  4.1267e-03,  1.6941e-02,  4.0744e-03,
         -6.9841e-03, -4.4967e-03],
        [ 2.7343e-02, -3.6506e-03,  1.3050e-02, -5.1261e-02, -6.3347e-03,
          1.2635e-02, -1.7651e-02],
        [-2.4080e-02,  1.7868e-02, -2.7517e-02, -6.6756e-03,  5.0212e-04,
          6.9584e-02, -4.3436e-02],
        [-3.5369e-02, -8.3450e-03,  1.9156e-02, -2.9539e-02,  3.1964e-02,
         -1.7826e-02, -4.1685e-02]], grad_fn=<SelectBackward0>)